In [1]:
from wandb.keras import WandbCallback
import sys
import time
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from utils.utils_mine import *
import tensorflow as tf
import tensorflow.keras as keras
from datetime import date
import numpy as np
import random
import tensorflow_addons as tfa
import wandb
import config
import gc
from classifiers.classifier_factory import create_classifier
from scripts.plot.DL.read_LOO_nestedCV_gnntr import get_sorted_loo_array




/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.14.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issu

In [4]:
'''/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_0/stratified_nested_5_CV_fold-0    '''


def get_sorted_loo_array(model, model_params):
    ALL_TOTAL_ITERATION = []
    TOTAL_Subject = 65
    K_FOLD = 5
    validation_method_external = 'LOO_nested_CV'
    validation_method_inner = 'stratified_nested_5_CV_fold'
    DATASET = 'prognosis/pre_treatment_hamd_reduction_50'
    RESULT_FILE_NAME = 'val_acc.txt'
    val_fold_path = f'results/{model}/{DATASET}/{model_params}/{validation_method_external}'
    total_subjects  = 46 if DATASET[:8] == 'pre_post' else TOTAL_Subject # '65' or '46


    for subject in range(total_subjects):
        for fold in range(K_FOLD):
            fold_path = f'{val_fold_path}/LOO_{subject}/{validation_method_inner}-{fold}'
            try:
                with open(f'{fold_path}/{RESULT_FILE_NAME}', 'r') as f:
                    best_iteration = int(f.read())
                    ALL_TOTAL_ITERATION.append(best_iteration)
            except:
                print(f'Error: {fold_path}/best_iteration.txt not found')
            
    loo_toal_itr = np.array(ALL_TOTAL_ITERATION).copy()
    loo_toal_itr = loo_toal_itr.reshape(-1, 5)
    loo_toal_itr = np.mean(loo_toal_itr, axis=1)
    sorted_indices = np.argsort(loo_toal_itr)
    sorted_indices = sorted_indices.tolist()
    print("Sorted indices:", sorted_indices, "Sorted values:", loo_toal_itr[sorted_indices])
    return sorted_indices



'/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'

In [20]:

model = 'yu_gnn'
model_params = 'v1'

def avg_total_itr_for_each_fold(ALL_TOTAL_ITERATION):
        loo_toal_itr = np.array(ALL_TOTAL_ITERATION).copy()
        loo_toal_itr = loo_toal_itr.reshape(-1, 5)
        loo_toal_itr = np.mean(loo_toal_itr, axis=1)
        return loo_toal_itr
def get_sorted_loo_array(model, model_params):

    ALL_TOTAL_ITERATION = [] # store all the total iteration for each fold
    TOTAL_Subject = 65 # number of subjects in the dataset for LOOCV in external testing set
    K_FOLD = 5 # number of k folds in inner CV
    validation_method_external = 'LOO_nested_CV' # external validation method
    validation_method_inner = 'stratified_nested_5_CV_fold' # inner validation method
    DATASET = 'prognosis/pre_treatment_hamd_reduction_50' # dataset name
    RESULT_FILE_NAME = 'val_acc.txt' # result file name
    val_fold_path = f'results/{model}/{DATASET}/{model_params}/{validation_method_external}'
    total_subjects  = 46 if DATASET[:8] == 'pre_post' else TOTAL_Subject # '65' or '46


    for subject in range(total_subjects):
        for fold in range(K_FOLD):
            fold_path = f'{val_fold_path}/LOO_{subject}/{validation_method_inner}-{fold}'
            try:
                with open(f'{fold_path}/{RESULT_FILE_NAME}', 'r') as f:

                    total_lines = len(f.readlines())
                    ALL_TOTAL_ITERATION.append(total_lines)
            except:
                # if the fold has not been created yet, then the total iteration is 0
                print('fold_path', fold_path)
                ALL_TOTAL_ITERATION.append(0)
                print(f'{fold_path}/{RESULT_FILE_NAME} will be set to 0 because it has not been trained yet')
    # average the total iteration for each fold
    loo_toal_itr = avg_total_itr_for_each_fold(ALL_TOTAL_ITERATION)
    sorted_indices = np.argsort(loo_toal_itr)
    sorted_indices = sorted_indices.tolist()
    
    # print("Sorted indices:", sorted_indices, "Sorted values:", loo_toal_itr[sorted_indices])
    return sorted_indices

sorted_indices = get_sorted_loo_array(model, model_params)
print(sorted_indices)

fold_path results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_5_CV_fold-0
results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_5_CV_fold-0/val_acc.txt will be set to 0 because it has not been trained yet
fold_path results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_5_CV_fold-1
results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_5_CV_fold-1/val_acc.txt will be set to 0 because it has not been trained yet
fold_path results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_5_CV_fold-2
results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_5_CV_fold-2/val_acc.txt will be set to 0 because it has not been trained yet
fold_path results/yu_gnn/prognosis/pre_treatment_hamd_reduction_50/v1/LOO_nested_CV/LOO_64/stratified_nested_

In [13]:
import re

def convert_to_float(value):
    value = np.array(value)
    try:
        final_value = value.astype(float)
        np.nan_to_num(final_value, nan=0.0)
    except ValueError:
        final_value = 0.0
    return final_value


def read_file_metric_y_pred(path):
    pattern = r"Y_pred_in_test: \[\[(.*?)\]\]"
    with open(path, 'r') as f:
        content = f.read()
        y_pred = re.findall(pattern, content)
    numbers_list = np.array([float(num) for s in y_pred for num in s.split()]).reshape(-1, 2).tolist()

    return numbers_list

str_list = read_file_metric_y_pred('/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/results/gnn_transformer/prognosis/pre_treatment_hamd_reduction_50/v2_repeat_3l1_rate_0.01_l2_rate_0.01_d_model_16_batch_size_64_n_layers_6/LOO_nested_CV/LOO_0/stratified_nested_5_CV_fold-1/test_acc.txt')
print(str_list)

[[0.42887446, 0.57112557], [0.8382789, 0.16172114], [0.5024056, 0.4975944]]


In [11]:
numbers_list = np.array([float(num) for s in str_list for num in s.split()]).reshape(-1, 2).tolist()
print(numbers_list)

[[0.42887446, 0.57112557], [0.8382789, 0.16172114], [0.5024056, 0.4975944]]


In [2]:
model_name = 'gnn_transformer'
params = 'v2_repeat_1l1_rate_0.01_l2_rate_0.01_d_model_16_batch_size_64_n_layers_6'


x = get_sorted_loo_array(model_name, params)

val_fold_path results/gnn_transformer/prognosis/pre_treatment_hamd_reduction_50/v2_repeat_1l1_rate_0.01_l2_rate_0.01_d_model_16_batch_size_64_n_layers_6/LOO_nested_CV/LOO_0
num_of_cv_folds 5
y_test [0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1]
MAX_ITR: 999 ranging ( 3 ~ 10 )
Model name: gnn_transformer
| Model Name | Testing Set |             |             |             | Validation Set |             |             |             |
|------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
|            | Balanced Accuracy | Sensitivity | Specificity | F1 Score | Balanced Accuracy | Sensitivity | Specificity | F1 Score |
| gnn_transformer   | 65.0000  | 40.0000  | 90.0000  | 46.1538  | 83.2257  | 76.4617  | 89.9898  | 86.8218  |
Sorted indices: [48, 33, 52, 51, 50, 49, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 0, 31, 54, 55, 56, 57, 58, 5

In [7]:
print(x)
x[0] = 48
update_config_file('LOO_ARRAY', x)

[48, 33, 52, 51, 50, 49, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 0, 31, 54, 55, 56, 57, 58, 59, 60, 61, 62, 47, 30, 64, 28, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 29, 15, 17, 27, 26, 25, 24, 16, 22, 23, 19, 18, 53, 20, 35, 34, 42, 32, 63, 21]
